# 1 - First run with the Discovery API
The key thing to investigate when starting to work with an API is the documentation. This should provide all the required information to start building requests: the base URL, endpoints, parameters, etcetera. For Discovery, the main body of documentation is the [API sandbox](https://discovery.nationalarchives.gov.uk/API/sandbox/index). This page provides a list of available endpoints and details on how to use them. 
Here, we will begin our investigation of the API by looking at the search endpoint.

## 1.1 - Making a request via the Sandbox
The Discovery sandbox allows you to build and test requests in the browser; a useful tool for learning how to use the API, which we will demonstrate here.

Figure 1 shows the search endpoint, expanded to show the details and with the search term "ajax". 
- Section 1 provides a model response from the endpoint, providing details such as field and value types. 
- Section 2 lists all parameters available on the endpoint, with a description of what each query parameter should be used for, and what type of value should be provided. Note that in the screenshot below, a considerable chunk of this section has been cropped out - the notebook would be too long if we included it all!
- Section 3 lists the response status codes you may expect from the endpoint.
- If you press the "Try it out!" button, subsequent sections are revealed. 
- The first new section (4) show the URL that was built with the query parameters and endpoint, generating a request to the API; and an example cURL request sending the request to the API.
- The final section (5) show the response from the API, separated into the response code, headers, and the response body.

![Discovery Sandbox](images/discovery_search.png)

**Figure 1** - The Discovery API Sandbox. Note that section 2 has been cropped to save space.

At this stage, have a go! In particular, try searching for a specific term, and make a note of the response to compare with the response we get when we make the same request with Python.
This documentation provides all the information we need to start building our own requests.

## 1.2 - Making a request via Python
We will start by repeating the request made in the sandbox to ensure we get the same response. 
The first step is to install and import the required packages, then we can start building our request. If you had a go with the Discovery Sandbox, try changing the search query below to the same one you used.

In [ ]:
%pip install -q requests
%pip install -q json
import requests
import json

In [ ]:
base_discovery_url = "https://discovery.nationalarchives.gov.uk/API"

search_endpoint = "/search/records"

search_query_parameter = "sps.searchQuery"

search_query = "ajax"

full_search_url = base_discovery_url + search_endpoint + "?" + search_query_parameter + "=" + search_query

response = requests.request("GET", full_search_url)

## First, make sure we got a 200 response

print(response.status_code)

In [ ]:
## Using the json.dumps() method, we can print the response nicely

print(json.dumps(response.json(), indent=4))

Success! The response we get from the API is the same as the response we got from the sandbox.
This response is a JSON object, which is full of information, but also quite long. We can isolate the sections we may be interested in to get a more manageable view. 
The results are a list of records under the "records" key and stored as a list; if we want to find the first one, we can specify it like so (remember that Python starts counting from 0, so the first record is at index 0):

In [ ]:
first_result = response.json()["records"][0]

print(json.dumps(first_result, indent=4))

If we wanted to view the total number of results, and the split between records held by TNA vs other archives; we can isolate those two values from the response and print them together.

In [ ]:
total_results = response.json()["count"]
tna_oth_split = response.json()["heldByReps"]
result_count_stats = {
    "total_results": total_results,
    "tna_results": tna_oth_split[0],        # The API returns the TNA results first
    "other_results": tna_oth_split[1]
}
print(json.dumps(result_count_stats, indent=4))

## 1.3 - Requests to a specific record series

With the number of results found, it may be beneficial to narrow our search to specific record series. Record series are the way TNA organises its records. [At the top level](https://discovery.nationalarchives.gov.uk/browse), records are divided by [governmental department](https://discovery.nationalarchives.gov.uk/browse/r/h/C1) (if held by TNA) or by [external archive](https://discovery.nationalarchives.gov.uk/browse/r/A). For example, records from the Admiralty are held under [ADM](https://discovery.nationalarchives.gov.uk/browse/r/h/C4), while [MCA](https://discovery.nationalarchives.gov.uk/browse/r/h/C412) holds the records of the Maritime and Coastguard Agency. Within these departments, records are divided into numeric series, collecting offices within a department (for example). Records can then be further divided into sub-series (several times). 

Helpfully, TNA has provided research guides for many of the record series it holds, which can be found on the [TNA website](https://www.nationalarchives.gov.uk/help-with-your-research/research-guides/). These guides cover a wide range of topics, providing good overviews of the records held by TNA, including important record series. To find information about Royal Navy ships called "Ajax", then the guide on [how to find logs and records from Royal Navy ships](https://www.nationalarchives.gov.uk/help-with-your-research/research-guides/royal-navy-ships-voyages-log-books/) will be particularly helpful. From this, we can create a list of record series. The list is available in the [additional resources](./aditional_data.py).

From the documentation, we can see that record series are included with the "sps.recordSeries" parameter. When testing with the sandbox, we can see that if we add multiple record series, the sandbox creates a URL with a new "sps.recordSeries" parameter for each record series.

With this information, we can write the code block below to build the API query. 


In [ ]:
import aditional_data

record_series = aditional_data.admiralty_record_series

record_series_url = base_discovery_url + search_endpoint + "?" 

for record in record_series:                        # Loop through the list of record series and add them to the URL
    record_series_url += record + "&"

record_series_url += "&sps.searchQuery=ajax"

print(record_series_url)

Remember that the URL is URL-encoded, so spaces (for example) are replaced with "%20". We can see that each record series is passed as a separate parameter. 

So what does the response look like?

In [ ]:
record_series_response = requests.request("GET", record_series_url)

print(json.dumps(record_series_response.json(), indent=4))

That's a more approachable response, but what time period does this cover?

In [ ]:
record_series_response_time_period = record_series_response.json()["timePeriods"]

print(json.dumps(record_series_response_time_period, indent=4))

## 1.4 - Precision searching and getting the data we want

With the search endpoint, we can simple add more query parameters to refine our search. To filter for records that cover a specific time period, we can add "sps.dateFrom" and "sps.dateTo" (the dates the record was active). As per the documentation, these are passed as ISO 8601 formatted dates. Thus, to filter for records relating to the Napoleonic wars, we just need to add these filters and cover the early 1800's. 

In [ ]:
date_from = "1800-01-01"
date_to = "1840-01-01"

record_series_url += "&sps.dateFrom=" + date_from + "&sps.dateTo=" + date_to

print(record_series_url)

In [ ]:
nepolionic_response = requests.request("GET", record_series_url)

print(json.dumps(nepolionic_response.json(), indent=4))

With these filters, we can see that the number of results has dropped down to just 20, all held by TNA. Using the API has allowed us to filter to a manageable number of results with ease. But we can still improve the results we get.

If we wanted to have the records in chronological order, we can do that with another parameter "sps.sortByOption". In the sandbox, this has a dropdown rather than a free-text box, indicating that only the options listed are valid. The process of adding it to the request URL is the same. Here we are going to use `DATE_ASCENDING`, which orders the results by record start dates.

In [ ]:
record_series_url += "&sps.sortByOption=DATE_ASCENDING"

chronological_response = requests.request("GET", record_series_url)

print(json.dumps(chronological_response.json(), indent=4))

Now if we isolate out the results, we'll just have records from the right era, and from record series we are interested in.

Remember that there are many more query parameters available for use in the search endpoint, allowing you to filter and refine your results very precisely. In the next notebook, we will start to work with other endpoints, and with more of the data. This will allow us to start building up more complex requests and retrieving more complete data about each record.